<a href="https://colab.research.google.com/github/Hagar633/Machine-learning-/blob/main/analyzerver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y mediapipe
!pip install mediapipe==0.10.11
!pip install decord

!pip install git+https://github.com/apple/ml-depth-pro.git
!mkdir -p checkpoints
!wget https://huggingface.co/apple/DepthPro/resolve/main/depth_pro.pt -O checkpoints/depth_pro.pt


--2025-08-18 19:36:43--  https://huggingface.co/apple/DepthPro/resolve/main/depth_pro.pt
Resolving huggingface.co (huggingface.co)... 13.35.202.97, 13.35.202.40, 13.35.202.34, ...
Connecting to huggingface.co (huggingface.co)|13.35.202.97|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/c3/f3/c3f3eea459224d17357ca9be878608d1562e5fdd7e5dfaab42edb3dd15a6133a/3eb35ca68168ad3d14cb150f8947a4edf85589941661fdb2686259c80685c0ce?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27depth_pro.pt%3B+filename%3D%22depth_pro.pt%22%3B&Expires=1755549404&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1NTU0OTQwNH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2MzL2YzL2MzZjNlZWE0NTkyMjRkMTczNTdjYTliZTg3ODYwOGQxNTYyZTVmZGQ3ZTVkZmFhYjQyZWRiM2RkMTVhNjEzM2EvM2ViMzVjYTY4MTY4YWQzZDE0Y2IxNTBmODk0N2E0ZWRmODU1ODk5NDE2NjFmZGIyNjg2MjU5YzgwNjg1YzBjZT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9u

In [1]:
import torch
import depth_pro
model, transform = depth_pro.create_model_and_transforms()
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device).eval()
model = model.half().to(device).eval()
from pickle import FRAME
from google.colab.patches import cv2_imshow
from decord import VideoReader
import mediapipe as mp
import cv2
import numpy as np
from matplotlib import pyplot as plt
import statistics

video_path = "kid1.mp4"
vr = VideoReader(video_path)
frame_count = len(vr)
print(f"Total frames: {frame_count}")
output_path = "kid1_fin.mp4"

def approximate_intrinsics(width, height):
    c_x, c_y = width/2, height/2
    return float(c_x), float(c_y)


def pixel_to_camera_coords(x, y, Z, f_x, f_y, c_x, c_y):
   X = (x - c_x) * Z / f_x
   Y = (y - c_y) * Z / f_y
   return X, Y, Z

def stability_calc(frame, depth_map, fx,fy, results, c_x, c_y):
  points = {}
  mp_pose = mp.solutions.pose

  h, w, _ = frame.shape
  for id, lm in enumerate(results.pose_landmarks.landmark):
        cx, cy = int(lm.x * w), int(lm.y * h)

        cz = depth_map[cy, cx]
        X, Y, Z = pixel_to_camera_coords(cx, cy, cz, fx, fy, c_x, c_y)
        if id == mp_pose.PoseLandmark.LEFT_ANKLE.value:
            points["left_ankle"] = (X, Y, Z)
        elif id == mp_pose.PoseLandmark.RIGHT_ANKLE.value:
            points["right_ankle"] = (X, Y, Z)
        elif id == mp_pose.PoseLandmark.LEFT_HIP.value:
            points["left_hip"] = (X, Y, Z)
        elif id == mp_pose.PoseLandmark.RIGHT_HIP.value:
            points["right_hip"] = (X, Y, Z)

        if id in [mp_pose.PoseLandmark.LEFT_ANKLE.value,
                  mp_pose.PoseLandmark.RIGHT_ANKLE.value]:
            cv2.circle(frame, (cx, cy), 5, (0, 255, 0), cv2.FILLED)

  mp_draw.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)


  com_x = (points["left_hip"][0] + points["right_hip"][0]) / 2
  com_y = (points["left_hip"][1] + points["right_hip"][1]) / 2
  com_z = (points["left_hip"][2] + points["right_hip"][2]) / 2

  foot_midx = (points["left_ankle"][0] + points["right_ankle"][0]) / 2
  foot_midy = (points["left_ankle"][1] + points["right_ankle"][1]) / 2
  foot_midz = (points["left_ankle"][2] + points["right_ankle"][2]) / 2

  stance_width= abs(points["left_ankle"][0] -  points["right_ankle"][0])
  sway = abs(com_x - foot_midx)
  norm= sway/stance_width
  return norm

# Sample every 5 seconds (assuming 30 fps)
fps = 7
step = fps * 1
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True)
mp_draw = mp.solutions.drawing_utils

sample_frame = vr[0].asnumpy()
height, width, _ = sample_frame.shape
c_x,c_y= approximate_intrinsics(width, height)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
var = 0
norms = []
for i in range(0, frame_count, step):

    frame = vr[i].asnumpy()
    rgb = cv2.cvtColor(frame , cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)
    img = transform(rgb).unsqueeze(0).to(device).half()
    with torch.no_grad():
        pred = model.infer(img)
        depth_map = pred["depth"].squeeze().cpu().numpy()
        depth_map = cv2.resize(depth_map, (frame.shape[1], frame.shape[0]))
        intrinsics = pred["focallength_px"].cpu().numpy()
        if np.ndim(intrinsics) == 0:
          fx = fy = float(intrinsics)
        else:
          fx, fy = intrinsics

    if results.pose_landmarks:
      norm= stability_calc(frame, depth_map, fx,fy, results,c_x,c_y)
      norms.append(norm)
    if len(norms) > 1:
      var = statistics.variance(norms)
    print("stability is" , norm)
    cv2.putText(frame,
            f"Frame: {i} | Stability: {norm:.2f} | Variance: {var:.2f}",
            (10, 20),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (0, 0, 200),
            2,
            cv2.LINE_AA)
    out.write(frame)


# --- ADD 3-SECOND BLACK FRAME WITH FINAL TEXT ---
black_frame = np.zeros_like(frame)  # same size as video frames
avg_stability = np.mean(norms)
final_var = statistics.variance(norms) if len(norms) > 1 else 0

# write fps * 3 frames (3 seconds)
for _ in range(fps * 3):
    # make a copy so we don't overwrite black_frame itself
    frame_with_text = black_frame.copy()

    txt = f"Average Stability: {avg_stability:.2f} | Final Variance: {final_var:.2f}"
    cv2.putText(frame_with_text,
                txt,
                (5, height // 2),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (255, 255, 255),
                2,
                cv2.LINE_AA)

    out.write(frame_with_text)

out.release()
print(f"Video saved to {output_path}")




Total frames: 223
stability is 0.004023175328563197
stability is 0.2959222384142898
stability is 0.1961956863511506
stability is 0.39001124022224226
stability is 0.016216424213932087
stability is 0.4859964939243074
stability is 0.04131629679353538
stability is 0.6460986504176067
stability is 0.008657456654357997
stability is 0.6571123300397034
stability is 0.007500819250633762
stability is 1.5386070132881038
stability is 0.01665739464720938
stability is 0.21284124247780078
stability is 0.2187201242159825
stability is 0.3254788847007349
stability is 0.17471208863015758
stability is 0.19856884974326425
stability is 0.1990204406243529
stability is 0.22635766797849002
stability is 0.34788215163261904
stability is 0.023908314086395325
stability is 0.7566928033619718
stability is 0.021261387541210788
stability is 0.14786960029764395
stability is 0.07697401906543214
stability is 0.07244677954298295
stability is 0.6306775167067011
stability is 0.008173685225111336
stability is 0.67768788078904